In [ ]:
# # Shear experiment
# Attempt to reproduce Figure 8 in [1].
#
#
# > [1] Holzapfel, Gerhard A., and Ray W. Ogden.
#     "Constitutive modelling of passive myocardium:
#     a structurally based framework for material characterization.
#     "Philosophical Transactions of the Royal Society of London A:
#     Mathematical, Physical and Engineering Sciences 367.1902 (2009): 3445-3475.
#
import dolfin
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import pulse

In [ ]:
try:
    from dolfin_adjoint import (
        Constant,
        DirichletBC,
        Expression,
        UnitCubeMesh,
        interpolate,
    )
except ImportError:
    from dolfin import (
        Constant,
        DirichletBC,
        interpolate,
        Expression,
        UnitCubeMesh,
    )

In [ ]:
# Create mesh
N = 2
mesh = UnitCubeMesh(N, N, N)

In [ ]:
# Create a facet fuction in order to mark the subdomains
ffun = dolfin.MeshFunction("size_t", mesh, 2)
ffun.set_all(0)

In [ ]:
# Mark subdomains
xlow = dolfin.CompiledSubDomain("near(x[0], 0) && on_boundary")
xlow_marker = 1
xlow.mark(ffun, xlow_marker)

In [ ]:
xhigh = dolfin.CompiledSubDomain("near(x[0], 1.0) && on_boundary")
xhigh_marker = 2
xhigh.mark(ffun, xhigh_marker)

In [ ]:
ylow = dolfin.CompiledSubDomain("near(x[1], 0) && on_boundary")
ylow_marker = 3
ylow.mark(ffun, ylow_marker)

In [ ]:
yhigh = dolfin.CompiledSubDomain("near(x[1], 1) && on_boundary")
yhigh_marker = 4
yhigh.mark(ffun, yhigh_marker)

In [ ]:
# Collect the functions containing the markers
marker_functions = pulse.MarkerFunctions(ffun=ffun)

In [ ]:
# Create mictrotructure
V_f = dolfin.VectorFunctionSpace(mesh, "CG", 1)

In [ ]:
# Fibers
f0 = interpolate(Expression(("1.0", "0.0", "0.0"), degree=1), V_f)
# Sheets
s0 = interpolate(Expression(("0.0", "1.0", "0.0"), degree=1), V_f)
# Fiber-sheet normal
n0 = interpolate(Expression(("0.0", "0.0", "1.0"), degree=1), V_f)

In [ ]:
# Collect the mictrotructure
microstructure = pulse.Microstructure(f0=f0, s0=s0, n0=n0)

In [ ]:
# Create the geometry
geometry = pulse.Geometry(
    mesh=mesh,
    marker_functions=marker_functions,
    microstructure=microstructure,
)

In [ ]:
# Use the default material parameters
material_parameters = {"a": 2.28, "b": 9.726, "a_f": 1.685, "b_f": 15.779}

In [ ]:
# Create material
material = pulse.HolzapfelOgden(parameters=material_parameters)

In [ ]:
# Eff / Ess strain ratio
strain_ratio = Constant(1.0)

In [ ]:
# Create costants defined for the dirichlet BC
u0 = Constant(0.0)
x_strain = u0 * strain_ratio / 2
y_strain = u0 * (1 / strain_ratio) / 2

In [ ]:
# Make Dirichlet boundary conditions
def dirichlet_bc(W):
    V = W if W.sub(0).num_sub_spaces() == 0 else W.sub(0)
    return [
        DirichletBC(V.sub(0), Constant(-x_strain), xlow),
        DirichletBC(V.sub(0), Constant(x_strain), xhigh),
        DirichletBC(V.sub(1), Constant(-y_strain), ylow),
        DirichletBC(V.sub(1), Constant(y_strain), yhigh),
    ]

In [ ]:
# Collect Boundary Conditions
bcs = pulse.BoundaryConditions(dirichlet=(dirichlet_bc,))

In [ ]:
# Create problem
problem = pulse.MechanicsProblem(geometry, material, bcs)

In [ ]:
# Solve problem
fig, ax = plt.subplots(1, 2)
u0s = [0.045, 0.6, 0.8]
markers = ["^", "s", "o"]
for i, sr in enumerate([2.05, 1.02, 0.48]):
    strain_ratio.assign(sr)
    Effs = []
    Sffs = []
    Esss = []
    Ssss = []
    for xi in np.linspace(0, 0.1, 10):
        pulse.iterate.iterate(
            problem,
            u0,
            Constant(xi),
            reinit_each_step=True,
        )
        S = problem.SecondPiolaStress()
        E = problem.GreenLagrangeStrain()

        Sff = dolfin.assemble(dolfin.inner(f0, S * f0) * dolfin.dx)
        Eff = dolfin.assemble(dolfin.inner(f0, E * f0) * dolfin.dx)

        Effs.append(Eff)
        Sffs.append(Sff)

        Sss = dolfin.assemble(dolfin.inner(s0, S * s0) * dolfin.dx)
        Ess = dolfin.assemble(dolfin.inner(s0, E * s0) * dolfin.dx)

        Esss.append(Ess)
        Ssss.append(Sss)

    ax[0].plot(Effs, Sffs, marker=markers[i], label=f"SR: {sr:.2f}")
    ax[1].plot(Esss, Ssss, marker=markers[i], label=f"SR: {sr:.2f}")
ax[0].set_ylabel("$S_{ff}$ (kPa)")
ax[1].set_ylabel("$S_{ss}$ (kPa)")

In [ ]:
ax[0].set_xlabel("$E_{ff}$")
ax[1].set_xlabel("$E_{ss}$")
for axi in ax:
    axi.legend()
fig.savefig("shear_test")